# 02 - Preprocessing Teks untuk TF-IDF

Pada tahap ini dilakukan preprocessing khusus untuk pemodelan berbasis **TF-IDF**.  
Berbeda dengan pemodelan berbasis **IndoBERT**, pada TF-IDF teks perlu dibuat lebih "ringkas" dan seragam.

## 02.1 - Tujuan Preprocessing TF-IDF

Preprocessing dilakukan untuk:

- mengurangi noise pada teks,
- menyederhanakan variasi kata,
- menurunkan dimensi kosakata (vocabulary),
- meningkatkan kinerja model klasik seperti **Logistic Regression**, **Linear SVM**, dan **Multinomial Naive Bayes**.

## 02.2 - Langkah-langkah Preprocessing

Langkah yang digunakan dalam penelitian ini untuk teks TF-IDF adalah:

1. **Case Folding**  
   Mengubah seluruh teks menjadi huruf kecil (lowercase).

2. **Penghapusan Noise Teknis**  
   - Menghapus URL (contoh: `https://...`, `http://...`)  
   - Menghapus mention `@username`  
   - Menghapus simbol `#` pada hashtag  
   - Menghapus angka dan karakter non-huruf

3. **Penghapusan Tanda Baca dan Emoji**  
   Tanda baca, emoji, dan simbol lain dihilangkan karena TF-IDF hanya memanfaatkan representasi kata.

4. **Tokenisasi**  
   Memecah kalimat menjadi deretan kata (token) berdasarkan spasi.

5. **Stopword Removal**  
   Menghapus kata-kata umum yang kurang informatif (misal: "yang", "dan", "atau", "di", "ke") menggunakan daftar stopword Bahasa Indonesia.

6. **Stemming**  
   Mengubah kata berimbuhan menjadi kata dasar menggunakan stemmer Bahasa Indonesia (misal: Sastrawi).  
   Contoh: *"menjalankan" → "jalan"*, *"korupsi" → "korupsi"*.

7. **Rejoin**  
   Menggabungkan kembali token hasil bersih menjadi string satu baris yang siap diolah oleh TF-IDF.


-----

## Import Library

In [1]:
import pandas as pd
import string
import nltk
import re
import requests
from pathlib import Path
from io import BytesIO
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

## Import Dataset

In [2]:
df = pd.read_csv("..\data\labeled_dataset_whoosh.csv")

In [3]:
df.head()

,video_id,video_title,comment_id,author,comment,likes,published_at,sentiment
0,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugy77mNqV3jn_Xf_f2x4AaABAg,@mokohardono8859,Yg benci ya apa aja salah.. \nYg seneng ya mak...,0,2025-12-03T05:33:53Z,neutral
1,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzY1qxEeVrMeskZ39V4AaABAg,@Muhidin13-kr1zl,Bandung akan miliki kereta pajajaran dgn beaya...,0,2025-12-02T09:48:07Z,neutral
2,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzlM3ejBnN9PKqB2sB4AaABAg,@MA_Alpha-l4q,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,1,2025-12-01T06:59:37Z,negative
3,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzcxKsZ3V_n222CvEF4AaABAg,@nurhasanahssi2114,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",0,2025-11-30T01:18:59Z,negative
4,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugx1UtqOkCAflakGpoh4AaABAg,@omsimon-k6k,Yg ditangkap gorengan yg makan duduk manis,0,2025-11-28T15:56:25Z,neutral


## Cek info kolom dan tipe data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1000 non-null   object
 1   video_title   1000 non-null   object
 2   comment_id    1000 non-null   object
 3   author        1000 non-null   object
 4   comment       1000 non-null   object
 5   likes         1000 non-null   int64 
 6   published_at  1000 non-null   object
 7   sentiment     1000 non-null   object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


## Cek beberapa contoh komentar dan label sentimen

In [5]:
df[["comment", "sentiment"]].head(10)

,comment,sentiment
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,neutral
1,Bandung akan miliki kereta pajajaran dgn beaya...,neutral
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,negative
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",negative
4,Yg ditangkap gorengan yg makan duduk manis,neutral
5,buat bayar hutang whossh jual saja Aset tentar...,negative
6,Pantas saja ngotot bgt lanjut 3 periode ternya...,negative
7,Analisa proyek.bukan kebutuhan.tapi keinginan ...,negative
8,This train won't make profit for 40 years at l...,negative
9,"“KERETA YANG BERLARI, NEGARA YANG TERSANDUNG”*...",negative


## Normalisasi nilai label (antisipasi huruf besar/kecil & bahasa campur)

In [6]:

df["sentiment"] = df["sentiment"].astype(str).str.strip().str.lower()

label_map_normalize = {
    "positif": "positive",
    "positif ": "positive",
    "positif👍": "positive",
    "positive": "positive",

    "negatif": "negative",
    "negatif ": "negative",
    "negative": "negative",

    "netral": "neutral",
    "neutral": "neutral",
}

df["sentiment"] = df["sentiment"].map(label_map_normalize).fillna(df["sentiment"])

df["sentiment"].value_counts(dropna=False)

sentiment
negative    646
neutral     320
positive     34
Name: count, dtype: int64

## Buang baris yang label-nya tidak valid

In [7]:
valid_labels = {"positive", "negative", "neutral"}

mask_valid = df["sentiment"].isin(valid_labels)
print("Baris valid:", mask_valid.sum())
print("Baris tidak valid (akan dibuang):", (~mask_valid).sum())

df = df[mask_valid].reset_index(drop=True)
df["sentiment"].value_counts()

Baris valid: 1000
Baris tidak valid (akan dibuang): 0


sentiment
negative    646
neutral     320
positive     34
Name: count, dtype: int64

## Encode label ke bentuk numerik

In [8]:
label_to_int = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

df["label"] = df["sentiment"].map(label_to_int)

df[["sentiment", "label"]].head()

,sentiment,label
0,neutral,1
1,neutral,1
2,negative,0
3,negative,0
4,neutral,1


In [9]:
df["label"].value_counts().sort_index().to_frame("count")

,count
label,
0,646
1,320
2,34


## Cek missing value

In [10]:
df.isna().sum()

video_id        0
video_title     0
comment_id      0
author          0
comment         0
likes           0
published_at    0
sentiment       0
label           0
dtype: int64

In [11]:
df  = pd.DataFrame(df[['comment', 'sentiment', 'label']])
df.head(5)

,comment,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",negative,0
4,Yg ditangkap gorengan yg makan duduk manis,neutral,1


## Proses Hapus Data Duplikat

In [12]:
df.drop_duplicates(subset ="comment", keep = 'first', inplace = True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 993 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    993 non-null    object
 1   sentiment  993 non-null    object
 2   label      993 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 31.0+ KB


## Proses Cleaning

In [14]:
# Fungsi untuk menghapus URL
def remove_URL(tweet):
    if tweet is not None and isinstance(tweet, str):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'', tweet)
    else:
        return tweet

# Fungsi untuk menghapus HTML
def remove_html(tweet):
    if tweet is not None and isinstance(tweet, str):
        html = re.compile(r'<.*?>')
        return html.sub(r'', tweet)
    else:
        return tweet

# Fungsi untuk menghapus emoji
def remove_emoji(tweet):
    if tweet is not None and isinstance(tweet, str):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F700-\U0001F77F"  # alchemical symbols
            u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
            u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
            u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            u"\U0001FA00-\U0001FA6F"  # Chess Symbols
            u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
            u"\U0001F004-\U0001F0CF"  # Additional emoticons
            u"\U0001F1E0-\U0001F1FF"  # flags
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', tweet)
    else:
        return tweet

# Fungsi untuk menghapus simbol
def remove_symbols(tweet):
    if tweet is not None and isinstance(tweet, str):
        tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    return tweet

# Fungsi untuk menghapus angka
def remove_numbers(tweet):
    if tweet is not None and isinstance(tweet, str):
        tweet = re.sub(r'\d', '', tweet)
    return tweet

# Fungsi hapus username
def remove_usernames(text):
    return re.sub(r'@\w+', '', text)

df['cleaning'] = df['comment'].apply(lambda x: remove_URL(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_usernames(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_html(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_emoji(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_symbols(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_numbers(x))

df = df[['comment', 'cleaning', 'sentiment', 'label']]

df.head(5)

,comment,cleaning,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,neutral,1


## Proses Case Folding

In [15]:
def case_folding(text):
    if isinstance(text, str):
        lowercase_text = text.lower()
        return lowercase_text
    else:
        return text

df['case_folding'] = df['cleaning'].apply(case_folding)

df = df[['comment', 'cleaning', 'case_folding' ,'sentiment', 'label']]

df.head(5)

,comment,cleaning,case_folding,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,neutral,1


## Normalisasi Kata

In [16]:
# Fungsi penggantian kata tidak baku
def replace_taboo_words(text, kamus_tidak_baku):
    if isinstance(text, str):
        words = text.split()
        replaced_words = []
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

        for word in words:
            if word in kamus_tidak_baku:
                baku_word = kamus_tidak_baku[word]
                if isinstance(baku_word, str) and all(char.isalpha() for char in baku_word):
                    replaced_words.append(baku_word)
                    kalimat_baku.append(baku_word)
                    kata_diganti.append(word)
                    kata_tidak_baku_hash.append(hash(word))
            else:
                replaced_words.append(word)
        replaced_text = ' '.join(replaced_words)
    else:
        replaced_text = ''
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

    return replaced_text, kalimat_baku, kata_diganti, kata_tidak_baku_hash

# Baca dataset kamu (pastikan df sudah tersedia)
data = pd.DataFrame(df[['comment','cleaning','case_folding', 'sentiment', 'label']])
data.head()

,comment,cleaning,case_folding,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,neutral,1


In [17]:
# Unduh dan baca kamus dari GitHub
url = "https://github.com/analysisdatasentiment/kamus_kata_baku/raw/main/kamuskatabaku.xlsx"
response = requests.get(url)
file_excel = BytesIO(response.content)
kamus_data = pd.read_excel(file_excel)

# Buat dictionary dari kamus
kamus_tidak_baku_dict = dict(zip(kamus_data['tidak_baku'], kamus_data['kata_baku']))

In [18]:
# Terapkan fungsi normalisasi
data[['normalisasi', 'Kata_Baku', 'Kata_Tidak_Baku', 'Kata_Tidak_Baku_Hash']] = data['case_folding'].apply(
    lambda x: pd.Series(replace_taboo_words(x, kamus_tidak_baku_dict))
)

# Ambil kolom yang relevan
df = pd.DataFrame(data[['comment','cleaning','case_folding','normalisasi', 'sentiment', 'label']])
df.head(5)

,comment,cleaning,case_folding,normalisasi,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,yang benci ya apa saja salah yang senang ya ma...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,sudah jelas geng solo yang harus bertanggung j...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yang harus bertanggungjawab...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,neutral,1


## Tokenization

In [19]:
def tokenize(text):
    tokens = text.split()
    return tokens

# Tambah kolom tokenize ke df
df['tokenize'] = df['normalisasi'].apply(tokenize)

# Ambil kolom yang relevan - gunakan 'df' bukan 'data'
df = df[['comment', 'cleaning', 'case_folding', 'normalisasi', 'tokenize', 'sentiment', 'label']]

df.head(5)

,comment,cleaning,case_folding,normalisasi,tokenize,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,yang benci ya apa saja salah yang senang ya ma...,"[yang, benci, ya, apa, saja, salah, yang, sena...",neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,"[bandung, akan, miliki, kereta, pajajaran, den...",neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,sudah jelas geng solo yang harus bertanggung j...,"[sudah, jelas, geng, solo, yang, harus, bertan...",negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yang harus bertanggungjawab...,"[jokowi, luhut, kroni, yang, harus, bertanggun...",negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,"[yang, ditangkap, gorengan, yang, makan, duduk...",neutral,1


## Proses Stopword Removal

In [20]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hafizh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df['stopword removal'] = df['tokenize'].apply(lambda x: remove_stopwords(x))

# Reorder kolom - tambahkan 'stopword removal' setelah 'tokenize'
df = df[['comment', 'cleaning', 'case_folding', 'normalisasi', 'tokenize', 'stopword removal', 'sentiment', 'label']]

df.head(5)

,comment,cleaning,case_folding,normalisasi,tokenize,stopword removal,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,yang benci ya apa saja salah yang senang ya ma...,"[yang, benci, ya, apa, saja, salah, yang, sena...","[benci, ya, salah, senang, ya, senang, nyinyir...",neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,"[bandung, akan, miliki, kereta, pajajaran, den...","[bandung, miliki, kereta, pajajaran, beaya, mu...",neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,sudah jelas geng solo yang harus bertanggung j...,"[sudah, jelas, geng, solo, yang, harus, bertan...","[geng, solo, bertanggung, tangkap]",negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yang harus bertanggungjawab...,"[jokowi, luhut, kroni, yang, harus, bertanggun...","[jokowi, luhut, kroni, bertanggungjawab, bayar...",negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,"[yang, ditangkap, gorengan, yang, makan, duduk...","[ditangkap, gorengan, makan, duduk, manis]",neutral,1


## Proses Stemming Data

In [22]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return [stemmer.stem(word) for word in text]

df['stemming_data'] = df['stopword removal'].apply(lambda x: ' '.join(stem_text(x)))

df = df[['comment', 'cleaning', 'case_folding', 'normalisasi', 'tokenize', 'stopword removal', 'stemming_data','sentiment', 'label']]

df.head(5)

,comment,cleaning,case_folding,normalisasi,tokenize,stopword removal,stemming_data,sentiment,label
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,Yg benci ya apa aja salah \nYg seneng ya makin...,yg benci ya apa aja salah \nyg seneng ya makin...,yang benci ya apa saja salah yang senang ya ma...,"[yang, benci, ya, apa, saja, salah, yang, sena...","[benci, ya, salah, senang, ya, senang, nyinyir...",benci ya salah senang ya senang nyinyir ya bah...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,"[bandung, akan, miliki, kereta, pajajaran, den...","[bandung, miliki, kereta, pajajaran, beaya, mu...",bandung milik kereta pajajaran beaya murah who...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,sudah jelas geng solo yang harus bertanggung j...,"[sudah, jelas, geng, solo, yang, harus, bertan...","[geng, solo, bertanggung, tangkap]",geng solo tanggung tangkap,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",Jokowi Luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yg harus bertanggungjawab ...,jokowi luhut kroni yang harus bertanggungjawab...,"[jokowi, luhut, kroni, yang, harus, bertanggun...","[jokowi, luhut, kroni, bertanggungjawab, bayar...",jokowi luhut kroni bertanggungjawab bayar huta...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,Yg ditangkap gorengan yg makan duduk manis,yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,"[yang, ditangkap, gorengan, yang, makan, duduk...","[ditangkap, gorengan, makan, duduk, manis]",tangkap goreng makan duduk manis,neutral,1


In [30]:
# Pastikan kolom stemming_data tidak berisi NaN atau string kosong
df["stemming_data"] = df["stemming_data"].astype(str)

# Ganti "nan" string atau kosong jadi NaN beneran
df["stemming_data"] = df["stemming_data"].replace(["nan", ""], pd.NA)

# Drop baris yang stemming_data-nya NaN
df = df.dropna(subset=["stemming_data"]).reset_index(drop=True)

print("Jumlah data setelah drop NaN di stemming_data:", len(df))

Jumlah data setelah drop NaN di stemming_data: 987


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   comment           987 non-null    object
 1   cleaning          987 non-null    object
 2   case_folding      987 non-null    object
 3   normalisasi       987 non-null    object
 4   tokenize          987 non-null    object
 5   stopword removal  987 non-null    object
 6   stemming_data     987 non-null    object
 7   sentiment         987 non-null    object
 8   label             987 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 69.5+ KB


## Simpan File CSV

In [32]:
df.to_csv("../data/dataset_preprocessed.csv", index=False)